In [79]:
from multirotor import MultiRotor

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import pandas as pd

import seaborn as sns

sns.set_style("whitegrid")

In [86]:
class mfac(nn.Module):
    def __init__(self, layer_sizes):
        super(mfac, self).__init__()
        
        self.first_layer = np.zeros(layer_sizes[0])
        
        self.hidden_layers = [
            nn.Sequential(nn.Linear(nlminus1, nl), nn.ReLU())
            for nl, nlminus1 in zip(layer_sizes[1:-1], layer_sizes)
        ]
        
        self.output_layer = nn.Linear(layer_sizes[-2], layer_sizes[-1])
        
    def forward(self, x):
        
        self.first_layer = np.roll(self.first_layer, 1)
        self.first_layer[0] = x
        first_layer_tensor = torch.from_numpy(self.first_layer)
        
        #HELP
        e = self.hidden_layers(first_layer_tensor)
        e = self.output_layer(e)
        
        return e

In [87]:
# Simulation timing
time_start = 0
time_stop = 30
num_time_points = 1000
time = np.linspace(time_start, time_stop, num_time_points)
time_step = time[1] - time[0]

In [88]:
mr = MultiRotor(dt=time_step)

# The input layer size
n0 = 5

# The output layer
nL = 1

# Hidden layers' sizes
neurons_per_layer = [13, 17]

# Preprend the input and append the output layer sizes
layer_sizes = [n0] + neurons_per_layer + [nL]

net = mfac(layer_sizes)

In [89]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0011)

In [90]:
# Altitude target (setpoint) is up 10m
altitude_setpoint = mr.get_altitude() + 10

In [91]:
for t in time:
    error = float(altitude_setpoint - mr.get_altitude())
    output = net(error)
    curr_altitude, curr_velocity = mr.step(output)
    
    optimizer.zero_grad()
    loss = criterion(output, torch.as_tensor([error]))
    loss.backward()
    optimizer.step()
    
    print(curr_altitude)

TypeError: 'list' object is not callable

In [ ]:
# frames.append(observation.copy())

# Initialize subplots
fig, ax = plt.subplots()

# Set axis of animation
ax.set(xlim=(0,300), ylim=(240,0))

# Initialize frame
frame = plt.imshow(frames[0])

def init():
    """
    Creates initial frame for animation.
    
    :return: (list) a list of the initial frame for animation
    """
    frame.set_data(frames[0])
    return [frame]

def animate(i):
    """
    Updates the state of the frame at each point of the animation.
    
    :param i: (ndarray) current image/frame
    :return: (list) a list of the current frame for animation
    """
    # Define the updated content of frame
    frame.set_array(i)
    return [frame]

from IPython.display import HTML

# Take images, functions, and other parameters and create an animation
ani = FuncAnimation(fig, animate, frames[::35], init_func = init, interval = 200)
HTML(ani.to_jshtml())
# -

# Save animation
ani.save("animation" + ".gif")